In [1]:
# 03_model_training.ipynb

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
import os

# Paths
TRAIN_DIR = "../data/raw/PlantVillage"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 15
MODEL_PATH = "../models/crop_disease_cnn.h5"
LABEL_PATH = "../models/label_encoder.pkl"

# ========================
# Step 1: Load Class Indices
# ========================
with open(LABEL_PATH, "rb") as f:
    class_indices = pickle.load(f)

num_classes = len(class_indices)
print("Number of classes:", num_classes)

# ========================
# Step 2: Data Generators
# ========================
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# ========================
# Step 3: Transfer Learning Model
# ========================
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ========================
# Step 4: Callbacks
# ========================
checkpoint = ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# ========================
# Step 5: Train
# ========================
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop]
)


Number of classes: 15
Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
9406464/9406464 [==============================] - 2s 0us/step
Epoch 1/15
517/517 [==============================] - ETA: 0s - loss: 0.7822 - accuracy: 0.7713
Epoch 1: val_accuracy improved from -inf to 0.84983, saving model to ../models\crop_disease_cnn.h5


c:\Users\Mukul\Desktop\Crop-Disease-Prediction\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


517/517 [==============================] - 608s 1s/step - loss: 0.7822 - accuracy: 0.7713 - val_loss: 0.4662 - val_accuracy: 0.8498
Epoch 2/15
517/517 [==============================] - ETA: 0s - loss: 0.3908 - accuracy: 0.8761
Epoch 2: val_accuracy improved from 0.84983 to 0.86536, saving model to ../models\crop_disease_cnn.h5
517/517 [==============================] - 488s 943ms/step - loss: 0.3908 - accuracy: 0.8761 - val_loss: 0.4011 - val_accuracy: 0.8654
Epoch 3/15
517/517 [==============================] - ETA: 0s - loss: 0.3259 - accuracy: 0.8931
Epoch 3: val_accuracy improved from 0.86536 to 0.89083, saving model to ../models\crop_disease_cnn.h5
517/517 [==============================] - 537s 1s/step - loss: 0.3259 - accuracy: 0.8931 - val_loss: 0.3370 - val_accuracy: 0.8908
Epoch 4/15
517/517 [==============================] - ETA: 0s - loss: 0.2749 - accuracy: 0.9128
Epoch 4: val_accuracy improved from 0.89083 to 0.90587, saving model to ../models\crop_disease_cnn.h5
517/517